##### Copyright 2019 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License").

In [0]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Non-rigid surface deformation
Non-rigid surface deformation is a technique that, among other things, can be used to interactively manipulate meshes or to deform a template mesh to fit to a point-cloud. When manipulating meshes, this can for instance allow users to move the hand of a character, and have the rest of the arm deform in a realistic manner. It is interesting to note that the deformation can also be performed over the scale of parts or the entire mesh.

![](https://storage.googleapis.com/tensorflow-graphics/notebooks/non_rigid_deformation/task.jpg)

This notebook illustrates how to use [Tensorflow Graphics](https://github.com/tensorflow/graphics) to perform defomations simliar to the one contained in the above image. 

## Setup & Imports
If Tensorflow Graphics is not installed on your system, the following cell can install the Tensorflow Graphics package for you.

In [0]:
!pip install tensorflow_graphics

Now that Tensorflow Graphics is installed, let's import everything needed to run the demo contained in this notebook.

In [0]:
import time

from google.colab.output import _js_builder
from google.colab.output import _publish
from matplotlib import collections
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from colabtools import adhoc_import
with adhoc_import.Google3():
  from tensorflow_graphics.geometry.deformation_energy import as_conformal_as_possible
  from tensorflow_graphics.geometry.transformation import quaternion
  from tensorflow_graphics.notebooks import threejs_visualization

In this example, we build a mesh that corresponds to a flat and rectangular surface. Using the sliders, you can control the position of the deformation constraints applied to that surface, which respectively correspond to all the points along the left boundary, center, and right boundary of the mesh.

In [0]:
###############
# UI controls #
###############
#@title Contraints on the deformed pose { vertical-output: false, run: "auto" }
constraint_1_z = 0  #@param { type: "slider", min: -1, max: 1 , step: 0.05 }
constraint_2_z = -1.0  #@param { type: "slider", min: -1, max: 1 , step: 0.05 }
constraint_3_z = 0  #@param { type: "slider", min: -1, max: 1 , step: 0.05 }

tf.reset_default_graph()


def _build_column(start_x, end_x, num_vertices_per_column, y):
  """For a fixed y, builds `num_vertices_per_column` vertices with x values
  varying between `start_x` and `end_x`. The z values are set to 0, except for
  the first, central, and last vertices that follow used-defined values."""
  column = np.zeros(shape=(num_vertices_per_column, 3))
  column[:, 0] = np.linspace(start_x, end_x, num_vertices_per_column)
  column[:, 1] = y
  column_rest_pose = np.array(column)
  column[0, 2] = constraint_1_z
  column[num_vertices_per_column / 2, 2] = constraint_2_z
  column[num_vertices_per_column - 1, 2] = constraint_3_z
  column_tf = tf.concat(
      (tf.constant(column[0:1, :]),
       tf.Variable(column[1:num_vertices_per_column / 2, :]),
       tf.constant(column[num_vertices_per_column /
                          2:num_vertices_per_column / 2 + 1, :]),
       tf.Variable(column[num_vertices_per_column / 2 +
                          1:num_vertices_per_column - 1, :]),
       tf.constant(
           column[num_vertices_per_column - 1:num_vertices_per_column, :])),
      axis=0)
  return column_tf, column_rest_pose


# Builds the mesh in the rest pose and a second mesh with the specified constraints.
def _create_stripe(num_vertices):
  num_vertices_per_column = num_vertices / 2
  start_x = -2.
  end_x = 2.
  # Builds vertices.
  stripe_width = 1.0
  left_column, left_column_rest = _build_column(start_x, end_x,
                                                num_vertices_per_column, 0.0)
  right_column, right_column_rest = _build_column(start_x, end_x,
                                                  num_vertices_per_column, 1.0)
  vertices_deformed = tf.concat((left_column, right_column), axis=0)
  vertices_rest = np.concatenate((left_column_rest, right_column_rest))

  # Builds faces and edges.
  faces = []
  edges = []
  for i in range(num_vertices_per_column):
    if i < num_vertices_per_column - 1:
      faces.append((i, i + 1, num_vertices_per_column + i))
      edges.append((i, i + 1))
      edges.append((i, num_vertices_per_column + i))
      edges.append((i + 1, num_vertices_per_column + i))
    if i > 0:
      faces.append((i, num_vertices_per_column + i,
                    num_vertices_per_column + i - 1))
      edges.append((i, num_vertices_per_column + i - 1))
      edges.append((i, num_vertices_per_column + i))
      edges.append((num_vertices_per_column + i - 1,
                    num_vertices_per_column + i))
  return vertices_rest, vertices_deformed, np.array(faces), list(
      dict.fromkeys(edges))


# Creating the rest pose and the rest pose with added constraints (deformed pose).
num_vertices = 38
vertices_rest_pose, vertices_deformed_pose, faces, connectivity = _create_stripe(
    num_vertices)
rotations = tf.Variable(quaternion.from_euler(np.zeros((num_vertices, 3))))

energy = as_conformal_as_possible.energy(vertices_rest_pose,
                                         vertices_deformed_pose, rotations,
                                         connectivity)

init = tf.initialize_all_variables()

sess = tf.Session()
sess.run(init)

mesh_rest_pose = {'vertices': vertices_rest_pose, 'faces': faces}
mesh_deformed_pose = {
    'vertices': sess.run(vertices_deformed_pose),
    'faces': faces
}

# Builds a camera and render the mesh.
camera = threejs_visualization.build_perspective_camera(
    field_of_view=40.0, position=(0.0, -5.0, 5.0))
width = 500
height = 500
threejs_visualization.triangular_mesh_renderer([mesh_rest_pose],
                                               width=width,
                                               height=height,
                                               camera=camera)
threejs_visualization.triangular_mesh_renderer([mesh_deformed_pose],
                                               width=width,
                                               height=height,
                                               camera=camera)

geometries = threejs_visualization.triangular_mesh_renderer(
    [mesh_deformed_pose], width=width, height=height, camera=camera)

################
# Optimization #
################
with tf.name_scope('optimizer'):
  # A quasi-Newton optimizer is used here for reasons of convergence speed.
  optimizer = tf.contrib.opt.ScipyOptimizerInterface(
      energy, options={'maxiter': 4})

num_iterations = 20
for it in range(num_iterations):
  optimizer.minimize(sess)
  vertices_deformed_pose_ = sess.run(vertices_deformed_pose)
  geometries[0].getAttribute('position').copyArray(
      vertices_deformed_pose_.ravel().tolist())
  geometries[0].getAttribute('position').needsUpdate = True
  geometries[0].computeVertexNormals()
  time.sleep(0.1)